In [1]:
import cv2
import json
import numpy as np
from PIL import Image, ImageOps # Importa o ImageOps da Pillow

# --- CONFIGURAÇÕES ---
IMAGEM_BASE = "Base/Placa.jpg"
ARQUIVO_SAIDA_JSON = "area_selecionada.json"
LARGURA_MAXIMA_JANELA = 1280
ALTURA_MAXIMA_JANELA = 720

# --- Variáveis globais e funções (sem alteração) ---
points_originais = []
razao_escala = 1.0
clone_exibicao = None

def draw_polygon_feedback(img_exibicao):
    # ... (esta função não muda)
    if not points_originais: return img_exibicao
    points_exibicao = [(int(p[0] * razao_escala), int(p[1] * razao_escala)) for p in points_originais]
    for point in points_exibicao: cv2.circle(img_exibicao, point, 5, (0, 255, 0), -1)
    if len(points_exibicao) > 1: cv2.polylines(img_exibicao, [np.array(points_exibicao)], isClosed=False, color=(0, 255, 0), thickness=2)
    if len(points_exibicao) == 4: cv2.line(img_exibicao, points_exibicao[3], points_exibicao[0], (0, 255, 0), 2)
    return img_exibicao

def click_event_handler(event, x, y, flags, param):
    # ... (esta função não muda)
    global points_originais
    if event == cv2.EVENT_LBUTTONDOWN and len(points_originais) < 4:
        x_original = int(x / razao_escala)
        y_original = int(y / razao_escala)
        points_originais.append((x_original, y_original))

# --- LÓGICA PRINCIPAL (MODIFICADA) ---
try:
    # --- NOVA PARTE: Usa a Pillow para abrir e corrigir a orientação ---
    print(f"Carregando e corrigindo a orientação da imagem: {IMAGEM_BASE}")
    pil_image = Image.open(IMAGEM_BASE)
    
    # Esta é a linha mágica que lê o EXIF e rotaciona a imagem corretamente
    pil_image_corrigida = ImageOps.exif_transpose(pil_image)
    
    # Converte a imagem da Pillow para o formato do OpenCV (NumPy array)
    # A imagem agora está na orientação correta para o OpenCV usar
    image_original = cv2.cvtColor(np.array(pil_image_corrigida), cv2.COLOR_RGB2BGR)
    # --- FIM DA NOVA PARTE ---

    if image_original is None:
        raise ValueError("A imagem não pôde ser carregada.")
    
    h_orig, w_orig = image_original.shape[:2]
    razao_escala = min(LARGURA_MAXIMA_JANELA / w_orig, ALTURA_MAXIMA_JANELA / h_orig)
    if razao_escala > 1: razao_escala = 1
    w_exibicao, h_exibicao = int(w_orig * razao_escala), int(h_orig * razao_escala)
    clone_exibicao = cv2.resize(image_original, (w_exibicao, h_exibicao), interpolation=cv2.INTER_AREA)

except Exception as e:
    print(f"ERRO: Não foi possível carregar ou processar a imagem '{IMAGEM_BASE}'.")
    print(f"Detalhe do erro: {e}")
    exit()

cv2.namedWindow("Definir Area (Poligono)")
cv2.setMouseCallback("Definir Area (Poligono)", click_event_handler)

print("\n--- INSTRUÇÕES ---")
# ... (o resto do script continua igual)
while True:
    display_image = clone_exibicao.copy()
    display_image = draw_polygon_feedback(display_image)
    cv2.imshow("Definir Area (Poligono)", display_image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"): break
    if key == ord("r"): points_originais = []; print("Pontos resetados.")
    if key == ord("s"):
        if len(points_originais) == 4:
            data = {"points": points_originais}
            with open(ARQUIVO_SAIDA_JSON, "w") as f: json.dump(data, f, indent=4)
            print(f"Sucesso! A área foi salva em '{ARQUIVO_SAIDA_JSON}'")
            break
        else:
            print(f"AVISO: Você selecionou apenas {len(points_originais)}/4 pontos.")
cv2.destroyAllWindows()

Carregando e corrigindo a orientação da imagem: Base/Placa.jpg

--- INSTRUÇÕES ---
Sucesso! A área foi salva em 'area_selecionada.json'
